<a href="https://colab.research.google.com/github/jahelsantiago/chess-net/blob/master/Main_algorith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob 
import numpy as np
import chess.pgn

In [ ]:
paths = ["/content/drive/My Drive/universidad/Ingeniería de sistemas/Deep Learning /CCRL-404FRC.[288996].pgn"]
games = []
for path in paths:
    pgn = open(path)    
    while True:
        game= chess.pgn.read_game(pgn)
        if game is not None:
            games.append(game)
        else:
            break
games_moves = []
for game in games:
    moves = ""
    for move in game.mainline_moves():
        temp = str(move)
        temp = temp[0:2]+" "+temp[2:4]
        moves+=temp + " "
    games_moves.append(moves) 

In [ ]:
import tensorflow as tf
from tensorflow import keras as k
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Bidirectional, Dense, LSTM, Embedding, Dropout
from keras.models import Sequential
from keras import regularizers

In [ ]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(games_moves)
sequences = tokenizer.texts_to_sequences(games_moves)

word_index = tokenizer.word_index
vocab_size = int(len(word_index) + 1)

max_lenght = max([len(x) for x in sequences])

temp = []
for line in sequences:
    for i in range(2, len(line)):
        temp.append(line[:i])
    

sequences_paded = pad_sequences(temp, maxlen= max_lenght)

xs = sequences_paded[:,:-1]
train_labels = sequences_paded[:,-1]
ys = to_categorical(train_labels)

ValueError: ignored

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_lenght-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(int(vocab_size/2), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
 history = model.fit(xs, ys, epochs=100, verbose=1)